# SymPauli_expect

Expectation Value of a Pauli Sum is as the Trace of PauliMatrix * DensityMatrix

$ PauliSum = \sum_{i} \ p_i \ P_i $

$ DensityMatrix = \sum_{j k} \ l_j r_k \ | L_j > < R_k | $

Thus

$ ExpectationValue = Tr(\ \sum_{i j k} \ p_j l_j r_k \ P_i | L_j > < R_k | \ ) $

While $ P_i $ is a PauliOperator (matrix) hence $ P_i | L_j > $ is becoming a new vector, it then perform the outer product with $ < R_k | $

However, for finding the Trace, all you need to do is to sum the diagonal components of the matrix.

Therefore everything outside of diagonal path can be ignored.

Which leads to:

$ ExpectationValue = \sum_{i j k} \ p_j l_j r_k \ P_i | L_j > < R_k | \ [\ iff \ P_i | L_j > == R_k \ ] $

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
P = [0,1,2,3]
L = [0,1,0,1]
R = [0,1,1,0]

# First find out bitflip Pauli
bitflips = [ (p&1)^(p>>1) for p in P]
bitflips

# Then do P_i | L_j > to make new vector
PL = [p^l for p,l in zip(bitflips, L)]
PL

if any( pl^r for pl,r in zip(PL,R) ):
    print(f"diagonal > False")
else:
    print(f"diagonal > True")


[0, 1, 1, 0]

[0, 0, 1, 1]

diagonal > False


In [3]:
# Now we verify this by actually computing the matrix representation
import numpy as np
import numexpr as ne

P = np.array([0,1,2,3], dtype='uint8')
L = np.array([0,1,0,1], dtype='uint8')
R = np.array([0,1,1,0], dtype='uint8')

# First find out bitflip Pauli
bitflips = (P&1) ^ (P>>1)
bitflips

PL = bitflips ^ L
PL

# Since this a 4qubit system, lets expand the vector
ref = (2**np.arange(4))[::-1]
ref

PLvec = np.zeros(2**4, dtype='uint8')
PLval = ref[PL.astype(bool)]
PLval
PLval = PLval.sum()
PLval
PLvec[PLval] = 1
PLvec

Rvec = np.zeros(2**4, dtype='uint8')
Rval = ref[R.astype(bool)]
Rval
Rval = Rval.sum()
Rval
Rvec[Rval] = 1
Rvec

PLRmatrix = np.outer(PLvec, Rvec)
PLRmatrix

np.trace(PLRmatrix)

array([0, 1, 1, 0], dtype=uint8)

array([0, 0, 1, 1], dtype=uint8)

array([8, 4, 2, 1], dtype=int32)

array([2, 1], dtype=int32)

3

...

array([4, 2], dtype=int32)

6

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

0

# This proves the theory. So all you need to do is stack 3 for loops together.

Actually you can do 2 for loops.

Since the last step is to find if PL == R, we can do a hashtable check to accomplish this goal.

If there do no exist exact same state as PL in R set, automatically means that for every r in R we obtain zero.

In [4]:
# assume that you have already done encoding IXYZ into 0123
pauli_list = dict() # { binary pauli as tuple : coefficient }
left_state = dict() # { state as tuple : coefficient }
right_state = dict() # { state as tuple : coefficient }
expect_val = 0

# This is how you compute the coefficient for PL
# [ I X Y Z]
from operator import itemgetter as itg
from operator import mul
from functools import reduce
pl_coef = [None, None, [1j, -1j], [1, -1]]

for p, cp in pauli_list.items():
    bitflips = [ (p&1)^(p>>1) for p in P ]

    for l, cl in left_state.items():
        pl = [p^l for p,l in zip(bitflips, L)]

        # This also check whether there is a pl==r
        # At the sametime we can obtain the coefficient
        cr = right_state.get(pl, None)
        if cr is not None:
            expect_val +=  cp * cl * cr * reduce(mul, (pl_coef[p][l] for p,l in zip(p,l) if p>1))